In [5]:
from transformers import AutoTokenizer, AutoModelForCausalLM, GPT2Tokenizer, GPT2LMHeadModel
import torch

In [ ]:
# Load pre-trained model and tokenizer
model_name = "gpt2"  # or any compatible model
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

In [ ]:
prompt = 'Does the sun revolve around the earth?'
desired_token = 'yes'
input_ids = tokenizer(prompt, return_tensors="pt").input_ids
target_token_id = tokenizer.convert_tokens_to_ids(desired_token)

In [ ]:
outputs = model(input_ids)
logits = outputs.logits[:, -1, :]

In [ ]:
bias_value = 1000  # Adjust this to control the strength of biasing
logits[:, target_token_id] += bias_value

In [ ]:
probs = torch.softmax(logits, dim=-1)